<a href="https://colab.research.google.com/github/daquarti/AI/blob/main/Named_Entity_Recognition_CRF_Bidireccional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
path = "/content/gdrive/My Drive/NLP/antecedentes/"

In [ ]:
pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-6bxj5lz9
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-6bxj5lz9
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101064 sha256=8810902f64554d02ac104a0b46d8bfcb252687a6d4aa001f629d58986c6004c0
  Stored in directory: /tmp/pip-ephem-wheel-cache-5n6tsuc3/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [ ]:
from keras.models import load_model
from keras_contrib.layers import CRF
from keras_contrib.losses import  crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy


# To load the model
custom_objects={'CRF'                 : CRF,
              'crf_loss'            : crf_loss,
              'crf_viterbi_accuracy': crf_viterbi_accuracy}

# To load a persisted model that uses the CRF layer 
model = load_model(path + 'my_model_antecedentes_junio_20.h5', custom_objects = custom_objects)

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [ ]:
import numpy as np

In [ ]:
import json
dics = json.load(open(path + 'dic_predict_crf_june_19.json', 'r'))

In [ ]:
import tensorflow as tf

In [ ]:
word2idx = dics['word2idx']

In [ ]:
idx2tag = dics['idx2tag']

In [ ]:
tag2idx = dics['tag2idx']

In [ ]:
word_tokenizer = tf.keras.preprocessing.text.text_to_word_sequence 

In [ ]:
n_words = 99170

In [ ]:
def prediccion (oracion):
  from keras.preprocessing.sequence import pad_sequences
  oracion_split = word_tokenizer (oracion)
  oracion_index = []
  for w in oracion_split:
    try:
      oracion_index.append(word2idx[w])
    except:
      oracion_index.append(word2idx['que'])
  oracion_final = pad_sequences(maxlen=86, sequences=[oracion_index], padding="post",value=n_words - 1)
  return oracion_final, oracion_split

In [ ]:
def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[str(p_i)])
        out.append(out_i)
    etiquetas = out
    tupla = list (zip (x_predict[1], etiquetas[0]))
    return tupla
    

In [ ]:
def decoder (tupla):
  antecedente = []
  combinado = []
  for c, x in enumerate (tupla):
    if x[1] == 'B-antecedentes':
      try:
        if lista[c+1][1] != 'I-antecedentes':
          antecedente.append (x[0])
      except:
        pass
      try:
        if lista[c+1][1] == 'I-antecedentes' and lista[c+2][1] == 'I-antecedentes':
          first = x[0]
          second = lista[c+1][0]
          third = lista[c+2][0]
          total = first + " " + second + " " + third
          combinado.append (total)
      except:
        pass
      try:
        if lista[c+1][1] == 'I-antecedentes':
          first = x[0]
          second = lista[c+1][0]
          total = first + " " + second
          combinado.append (total)
      except:
        pass
        antecedente.append (x[0])
  final = list(set(antecedente + combinado))
  return final

In [ ]:
x_predict = prediccion ("PAciente hipertenso, obeso con antecedente de estenosis aórtica severa")

In [ ]:
test_pred = model.predict(x_predict[0], verbose=1)   

1/1 [==============================] - 0s 36ms/step


In [ ]:
lista = pred2label(test_pred)

In [ ]:
decoder(lista)

['epoc', 'dislipemia', 'dbt', 'estenosis aórtica', 'icc', 'obeso']

In [ ]:
lista 

[('paciente', 'O'),
 ('hipertenso', 'B-antecedentes'),
 ('obeso', 'B-antecedentes'),
 ('con', 'O'),
 ('antecedente', 'O'),
 ('de', 'O'),
 ('estenosis', 'B-antecedentes'),
 ('aórtica', 'I-antecedentes'),
 ('severa', 'O')]